# dynprops - Python Dynamic Properties
Support for complex table and TSV structures

## Revision History
* 0.0.1 - Initial commit

## Introduction
The `dynprops` package supports three basic features:
* Creation of sets properties that can be emitted in columnar form (tsv, csv, ...) or as dictionaries targeted at SQL tools
* Global 'singleton' properties for columns that are invariant over time
* "Dynamic properties" - properties whose values can either be objects, functions or methods

## Requirements
* The `dynobject` package requires Python 3.6 or later as it uses the 3.6 annotation features (`var: type [= value]`) 

## Installation


In [1]:
! pip install -e ..

  Using cached pip-9.0.2-py2.py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1
Obtaining file:///Users/mrf7578/Development/git/BD2KOnFHIR/dynprops
  Found existing installation: dynprops 0.1.0
    Uninstalling dynprops-0.1.0:
      Successfully uninstalled dynprops-0.1.0
  Running setup.py develop for dynprops


## Usage
### Declaring properties:

Properties are declared as types, using either `Global`, meaning that the property is a singleton or
`Local`, meaning that the property can be class or instance level.

In [27]:
from datetime import datetime
from dynprops import DynProps, Global, Local, Parent, heading, row, as_dict


# The base -- Three global properties - a string, an integer and a date with a dynamic(!) value
class BaseElements(DynProps):
    key_part1: Global[str]
    key_part2: Global[int]
    key_part3: Global[datetime] = datetime.now      # Function


Dynamic properties can be used to generate csv/tsv/?sv headings

In [28]:
DynProps._separator = ', '      # Default is tab
print(heading(BaseElements))


key_part1, key_part2, key_part3


An extension to the base with one more global property and two locals


In [29]:
class ShortTable(BaseElements):
    key_part4: Global[str]
    val_part1: Local[str] = "@"
    val_part2: Local[int] = 0

    def __init__(self, vp2: int) -> None:
        self.val_part2 = vp2
print(heading(ShortTable))  

key_part1, key_part2, key_part3, key_part4, val_part1, val_part2


We can now set the values of the global components. Globals must be set at the level of the declaring class
(i.e. `ShortTable.key_part2 = 1` or `BaseElements().key_part_2 = 1` will not work. 

The `row` function prints the value of an instance

In [30]:
BaseElements.key_part1 = "production run"
BaseElements.key_part2 = 17

ShortTable.val_part1 = "Unit"
print(row(ShortTable(3)))
print(row(ShortTable(4)))

"production run", 17, 2018-03-17 15:01:04.665801, , "Unit", 3
"production run", 17, 2018-03-17 15:01:04.667213, , "Unit", 4


Note that `key_part_3` is a function (`datetime.now`) so each row gets a new value

We can override the defaults with a fixed value:

In [31]:
BaseElements.key_part3 = datetime(2017, 1, 31)
print(row(ShortTable(143)))

"production run", 17, 2017-01-31 00:00:00, , "Unit", 143


The `_clear()` function resets everything *in the target class* back to defaults.

In [32]:
BaseElements._clear()
print(row(ShortTable(12)))

, , 2018-03-17 15:01:04.690232, , "Unit", 12


DynProps values can be fixed values, functions (`f() -> object`) or methods (`f(self) -> object`).  The first 
(and only) argument must be "self" in the second case.

The `Parent` tag identifies where the parent properties appear.  The default is before the children, but they can
appear after or even in the middle:

In [33]:
class MedTable(ShortTable):
    val_part3: Local[str] = "@"
    _: Parent
    val_part4: Local[int] = lambda self: self.val_part2 * 2

BaseElements.key_part1 = "Production Run"
BaseElements.key_part2 = 1001743
BaseElements.key_part3 = datetime.now
        
mt = MedTable(42)
print(heading(mt))
print(row(mt))
print(row(mt))

val_part3, key_part1, key_part2, key_part3, key_part4, val_part1, val_part2, val_part4
"@", "Production Run", 1001743, 2018-03-17 15:01:04.714837, , "Unit", 42, 84
"@", "Production Run", 1001743, 2018-03-17 15:01:04.716184, , "Unit", 42, 84


### Dynamic Properties as dictionaries
The `as_dict` function returns an ordered dictionary representation of a `DynProps` instance

In [34]:
for k, v in as_dict(mt).items():
    print(f"{k}: {v}")

val_part3: @
key_part1: Production Run
key_part2: 1001743
key_part3: 2018-03-17 15:01:04.726471
key_part4: None
val_part1: Unit
val_part2: 42
val_part4: 84


### Additional features
* **`DynProps._sql_string_delimiter`** - the delimiter to be used in `row` representation. Default: `'"'`
* **`DynProps._sql_string_delimiter_escape`** - escape character for an embedded string.  Default: `r'\"'`.  You may need
to set this to `'""'` for Oracle implementations
* **`DynProps._sql_null_text`** - representation for null (Python: `None`) in `row`.  Default is an empty string (`''`)
* **`DynProps._separator`** - `heading` and `row` separator.  Default is a tab (`'\t'`)

## Undone tasks

1) The typing within the `Global` and `Local` elements is not directly exposed.  We need to re-write the `__annotation__` elements in the constructor to allow IDE type checking.

2) Typing is generally ignored.  Strings are always quoted, integers are not.  We could get considerably more clever here

3) SQL value escaping is minimal -- the `row` functions, in particular, need to take full advantage of SQL library escaping

4) There are no SQL injection protections in this module. May not be needed, but should be at least discussed